In [2]:
# Import libraries and dependencies
import os
import pandas as pd
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation

In [3]:
# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)


In [31]:
# Format dates as ISO format
start1 = pd.Timestamp("2018-12-10", tz="America/New_York").isoformat()
end1 = pd.Timestamp("2020-01-20", tz="America/New_York").isoformat()
start2 = pd.Timestamp("2020-01-21", tz="America/New_York").isoformat()
today = pd.Timestamp("2021-09-20", tz="America/New_York").isoformat()

In [11]:
# Set the tickers
tickers = ["MRNA", "PFE","INO"]

In [12]:
# Set timeframe to one day ('1D') for the Alpaca API
timeframe = "1D"

In [32]:
# Get closing prices 
df_portfolio_all = alpaca.get_barset(
    tickers,
    timeframe,
    start = start1,
    end = today,
    limit = 1000
).df

# Display sample data
df_portfolio_all.head(10)

INO                                MRNA           \
                           open    high   low close   volume   open     high   
time                                                                           
2018-12-10 00:00:00-05:00  5.17  5.2200  5.02  5.20   794144  18.90  19.4388   
2018-12-11 00:00:00-05:00  5.25  5.3300  5.09  5.18   477373  20.55  20.6600   
2018-12-12 00:00:00-05:00  5.21  5.3800  5.21  5.30   623406  18.55  19.4800   
2018-12-13 00:00:00-05:00  5.31  5.3200  5.08  5.11   772978  19.25  19.2900   
2018-12-14 00:00:00-05:00  5.06  5.1000  4.84  4.87  1048009  18.75  18.8806   
2018-12-17 00:00:00-05:00  4.86  5.0099  4.72  4.73   784021  18.33  18.4700   
2018-12-18 00:00:00-05:00  4.77  4.8100  4.47  4.53   761845  17.10  17.2500   
2018-12-19 00:00:00-05:00  4.50  4.7600  4.40  4.45   818839  16.86  17.3700   
2018-12-20 00:00:00-05:00  4.48  4.5300  4.20  4.25  1062680  16.00  16.1400   
2018-12-21 00:00:00-05:00  4.27  4.2700  3.81  3.89  1561882  15.40  15.5000   

                                                      PFE                  \
                             low    close   volume   open    high     low   
time                                                                        
2018-12-10 00:00:00-05:00  18.00  18.7577  4155179  44.14  44.560  43.070   
2018-12-11 00:00:00-05:00  17.99  18.0100  3377151  44.08  44.510  43.570   
2018-12-12 00:00:00-05:00  18.02  18.7500  2597460  44.35  44.845  44.150   
2018-12-13 00:00:00-05:00  18.68  18.7600  1616914  44.46  44.900  44.210   
2018-12-14 00:00:00-05:00  18.25  18.3100   733696  44.08  44.120  43.425   
2018-12-17 00:00:00-05:00  16.90  17.0000  1489852  43.50  43.970  42.980   
2018-12-18 00:00:00-05:00  16.25  16.9000  1494066  43.47  43.500  42.040   
2018-12-19 00:00:00-05:00  16.00  16.0250   942887  42.83  42.990  41.560   
2018-12-20 00:00:00-05:00  14.75  15.5030  2365478  41.71  42.410  41.530   
2018-12-21 00:00:00-05:00  13.95  14.4350  2033655  41.64  42.870  41.450   

                                             
                            close    volume  
time                                         
2018-12-10 00:00:00-05:00  44.405  22466277  
2018-12-11 00:00:00-05:00  44.020  20020936  
2018-12-12 00:00:00-05:00  44.170  14819586  
2018-12-13 00:00:00-05:00  44.550  16251482  
2018-12-14 00:00:00-05:00  43.845  24085732  
2018-12-17 00:00:00-05:00  43.100  30524854  
2018-12-18 00:00:00-05:00  42.400  24825645  
2018-12-19 00:00:00-05:00  41.970  30307929  
2018-12-20 00:00:00-05:00  41.960  26909733  
2018-12-21 00:00:00-05:00  41.950  34505684

In [34]:
# Create an empty DataFrame for closing prices
df_closing_prices = pd.DataFrame()

# Fetch the closing prices 
df_closing_prices["MRNA"] = df_portfolio_all["MRNA"]["close"]
df_closing_prices["PFE"] = df_portfolio_all["PFE"]["close"]
df_closing_prices["INO"] = df_portfolio_all["INO"]["close"]

# Drop the time component of the date
df_closing_prices.index = df_closing_prices.index.date

# Display sample data
df_closing_prices.head(10)

,MRNA,PFE,INO
2018-12-10,18.7577,44.405,5.20
2018-12-11,18.0100,44.020,5.18
2018-12-12,18.7500,44.170,5.30
2018-12-13,18.7600,44.550,5.11
2018-12-14,18.3100,43.845,4.87
2018-12-17,17.0000,43.100,4.73
2018-12-18,16.9000,42.400,4.53
2018-12-19,16.0250,41.970,4.45
2018-12-20,15.5030,41.960,4.25
2018-12-21,14.4350,41.950,3.89


In [35]:
# Compute daily returns
df_daily_returns = df_closing_prices.pct_change().dropna()

# Display sample data
df_daily_returns.head()

,MRNA,PFE,INO
2018-12-11,-0.039861,-0.008670,-0.003846
2018-12-12,0.041088,0.003408,0.023166
2018-12-13,0.000533,0.008603,-0.035849
2018-12-14,-0.023987,-0.015825,-0.046967
2018-12-17,-0.071546,-0.016992,-0.028747


In [36]:
import hvplot.pandas

In [38]:
# Use hvplot() function to plot data
df_daily_returns.hvplot.line(xlabel="Year", ylabel="Daily Return")

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [40]:
#volumne
# Create an empty DataFrame for volumn
df_volumn = pd.DataFrame()

# Fetch the closing prices 
df_volumn["MRNA"] = df_portfolio_all["MRNA"]["volume"]
df_volumn["PFE"] = df_portfolio_all["PFE"]["volume"]
df_volumn["INO"] = df_portfolio_all["INO"]["volume"]

# Drop the time component of the date
df_volumn.index = df_volumn.index.date
df_volumn = df_volumn.dropna()
# Display sample data
df_volumn.head(10)

,MRNA,PFE,INO
2018-12-10,4155179,22466277,794144
2018-12-11,3377151,20020936,477373
2018-12-12,2597460,14819586,623406
2018-12-13,1616914,16251482,772978
2018-12-14,733696,24085732,1048009
2018-12-17,1489852,30524854,784021
2018-12-18,1494066,24825645,761845
2018-12-19,942887,30307929,818839
2018-12-20,2365478,26909733,1062680
2018-12-21,2033655,34505684,1561882


In [45]:
df_volumn.hvplot.line(rot=60)

:NdOverlay   [Variable]
   :Curve   [index]   (value)